# RNN (NLP) 과제

1. 이름 분류

2. 문자 분류

3. Seq2seq 번역

4. TRANSFORMER 구조 번역

# 1.이름 분류

https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

목적 : torchtext와 같은 라이브러리 사용안하고 low-level에서 NLP가 어떻게 작동하나 알아보자!

방법 : 이름이 character단위로 rnn model로 들어가고 hidden state와 다음 character가 다음 epoch에 model로 전달됨. 

        마지막 단계에선 class를 output으로 내보낸다.

        18개의 언어와 몇천개의 성(surnames)이 있는데 어느 나라에 속하는지 맞추는 문제!

[Datadown link](https://download.pytorch.org/tutorial/data.zip)에서 다운받은 파일은 각 국가별로 총 18개의 .txt파일로 존재한다.

각 [Language].txt파일에 이름이 line별로 적혀있음

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path) #path내에 존재하는 파일을 list로 반환

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"        # letters, 이름에 포함될 수 있는 character수는 a~z , A~Z , 공백.,;' 을 포함해서 총 57개
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):    #filename의 file을 열고 각 line의 값을 list로 반환
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names\\Arabic.txt', 'data/names\\Chinese.txt', 'data/names\\Czech.txt', 'data/names\\Dutch.txt', 'data/names\\English.txt', 'data/names\\French.txt', 'data/names\\German.txt', 'data/names\\Greek.txt', 'data/names\\Irish.txt', 'data/names\\Italian.txt', 'data/names\\Japanese.txt', 'data/names\\Korean.txt', 'data/names\\Polish.txt', 'data/names\\Portuguese.txt', 'data/names\\Russian.txt', 'data/names\\Scottish.txt', 'data/names\\Spanish.txt', 'data/names\\Vietnamese.txt']
Slusarski


이름을 Tensor 형태로 바꾼다.

one - hot 벡터형식으로 표현 (이름길이,1,letters개수 원-핫)

1차원은 model넣을 때 batch가 될 자리임

In [3]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


RNN 모델 설계

2개의 hidden layer

hidden state는 activation 함수 통과 x

마지막 output은 logsoftmax를 통과해 최종 output_size차원(18개 국가)로 나옴

In [7]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
print(f'input_size : {n_letters} , output_size : {n_categories}')

input_size : 57 , output_size : 18


Training

In [9]:
# 1 x n_categories 차원의 output벡터에서 가장 큰 값의 index와 해당하는 class를 반환 함
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i